# création du dataset #

## ce notebook est à utiliser dans le cadre d'un entrainement en remote sur base d'excels dejà sortis. dans le cas ci dessous les courbes de consomations viennent d'energis et le reste viennent du BMS (sorties à la main) ##

In [10]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
#from sympy.codegen import Print
import requests
# === Paramètres ===
file_path = ".\data\database.xlsx"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)
# === Chargement des données ===

df = pd.read_excel(file_path,'Feuil1')
file_conso_path = ".\data\THHeating.xlsx"
cg = pd.read_excel(file_conso_path)

file_conso_path = ".\data\TH.xlsx"
cw = pd.read_excel(file_conso_path)
print(df)
print(cg)
print(cw)

['Sheet1', 'Feuil1']


C:\Users\hugom\OneDrive\Documents\Stage_2025\Stage_env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\hugom\OneDrive\Documents\Stage_2025\Stage_env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


                Horodatage  \
0      18/08/2025 23:59:54   
1      18/08/2025 23:58:45   
2      18/08/2025 23:57:40   
3      18/08/2025 23:56:38   
4      18/08/2025 23:55:35   
...                    ...   
19852  07/07/2025 01:00:00   
19853  07/07/2025 00:45:00   
19854  07/07/2025 00:30:00   
19855  07/07/2025 00:15:00   
19856  07/07/2025 00:00:00   

       +2_Ss_ECS_Ret_Temp Interval Trend Log - Extended Trend Log  \
0                                              71.000000            
1                                              71.000000            
2                                              71.000000            
3                                              71.000000            
4                                              71.000000            
...                                                  ...            
19852                                          63.399994            
19853                                          63.399994            
19854             

In [11]:
data = []
k= 0
for i in df:
    data.append(df[i])
cmd = np.transpose(data)
print(data[1])
print(np.shape(cmd))
print(cmd[0])
#cmd = matrices des commandes en fonction du temps

0        71.000000
1        71.000000
2        71.000000
3        71.000000
4        71.000000
           ...    
19852    63.399994
19853    63.399994
19854    63.399994
19855    63.399994
19856    63.399994
Name: +2_Ss_ECS_Ret_Temp Interval Trend Log - Extended Trend Log, Length: 19857, dtype: float64
(19857, 16)
['18/08/2025 23:59:54' 71.0 73.8000183105469 66.8000183105469 74.0
 71.8000183105469 74.0 72.3999938964844 58.8000183105469 63.0
 73.2000122070313 78.6000061035156 0.50120002746582 0.6980999755859381
 0.699499969482422 0.0]


## création import API pour get température

In [12]:
import requests
import pandas as pd
from datetime import datetime
date = "01-01-24 09:00:00"
def get_temp(date_datetime,lat =50.8503, lon = 4.3517):
    date = str(date_datetime)
    date = date.split(" ")
    date = date[0]
    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={date}&end_date={date}"
        f"&hourly=temperature_2m"
    )

    response = requests.get(url)
    data = response.json()
    temps = data['hourly']['temperature_2m']
    timestamps = data['hourly']['time']
    df = pd.DataFrame({'time': timestamps, 'temperature': temps})
    df['time'] = pd.to_datetime(df['time'])

    return df['temperature']

date2 = datetime.strptime(date, "%d-%m-%y %H:%M:%S")
print(get_temp(date2))

0     7.9
1     6.3
2     7.1
3     7.2
4     6.8
5     6.8
6     6.6
7     6.3
8     6.0
9     6.3
10    7.5
11    8.2
12    9.0
13    7.9
14    7.9
15    7.6
16    7.1
17    7.1
18    6.9
19    7.0
20    7.1
21    7.4
22    7.7
23    7.8
Name: temperature, dtype: float64


## création classe segment d'input ##

In [13]:
class segment():
    def __init__(self,Cmd,temp_ext,time,water_cons,conso_list,pw):
        self.cmd = Cmd
        self.conso_list = conso_list.tolist()
        self.temp_ext = temp_ext
        self.time = time.tolist()
        self.water_cons = water_cons.tolist()
        self.pw = pw
        #self.occupation = occupation
    def gettime(self):

        return self.time
    def getdata(self):

        t = self.time[pw].hour
        conso = self.conso_list[pw]
        data = self.cmd,conso,self.temp_ext,self.water_cons,t
        return data
    def flatten(self):
        t = self.time[pw].hour
        data = []
        for i in self.cmd:
            for j in i:
                data.append(j)
        conso = self.conso_list[pw]
        data.append(conso)
        for i in self.temp_ext:
            data.append(i)
        data.append(self.water_cons)
        data.append(t)
        for i in self.conso_list:
            data.append(i)
        return data
    #idéalement j'aimerais rajouter l'occupation mais elle n'est pas encore comprise dans les trends

### égalisation échelles de temps ###

In [14]:
from datetime import datetime
from datetime import timedelta
from IPython.display import clear_output
import time


normalized_cmd=[]
drop = []
for i in range(0,len(cg['FIFTEEN_MINUTE']),1):
    # Deux dates au format chaîne de caractères
    min = timedelta(days=5, hours=0, minutes=0, seconds=0)
    for j in range(len(data[0])):
        date1 = cg['FIFTEEN_MINUTE'][i]
        data_temp = data[0][j].replace("/","-")
        data_temp = data_temp[:6]+data_temp[8:]
        date2 = datetime.strptime(data_temp, "%d-%m-%y %H:%M:%S")
        n_min = abs(date1- date2)
        if n_min<min:
            min = n_min
            indx = j
    clear_output()
    print(cg['FIFTEEN_MINUTE'][i])
    data_temp = data[0][indx].replace("/","-")
    data_temp = data_temp[:6]+data_temp[8:]
    print(datetime.strptime(data_temp, "%d-%m-%y %H:%M:%S"))
    print(abs(min))
    if min<timedelta(days=0, hours=0, minutes=5, seconds=0):
        normalized_cmd.append(cmd[indx])

    else:
        drop.append(i)
        print("🫠aucune correspondance trouvée")

    print(f"😑wala j'y travaille : {float(np.shape(normalized_cmd)[0]) / float(np.shape(cg['FIFTEEN_MINUTE'])[0]-len(drop))}")
for i in drop:
    cw = cw.drop(i).reset_index(drop=True)
    cg = cg.drop(i).reset_index(drop=True)

2025-08-17 23:45:00
2025-08-17 23:45:00
0 days 00:00:00
😑wala j'y travaille : 1.0


In [15]:
#verification des tailles
print(len(cg))
print(len(cw['FIFTEEN_MINUTE']))
print(cg['FIFTEEN_MINUTE'][0])
print(normalized_cmd[0])
normalized_cmd = np.transpose(np.transpose(normalized_cmd)[1:])
print(normalized_cmd[0])

3931
3931
2025-07-07 00:00:00
['07/07/2025 00:00:00' 63.3999938964844 66.2000122070313 60.6000061035156
 66.3999938964844 65.6000061035156 66.3999938964844 64.8000183105469
 60.2000122070313 65.2000122070313 65.6000061035156 72.3999938964844 nan
 nan nan nan]
['18/08/2025 23:59:54' 71.0 73.8000183105469 66.8000183105469 74.0
 71.8000183105469 74.0 72.3999938964844 58.8000183105469 63.0
 73.2000122070313 78.6000061035156 0.50120002746582 0.6980999755859381
 0.699499969482422 0.0]


### création des segments ###

In [17]:
temp_today = {}
n_data = []
pw = 19
fw = 1
for i in range(pw,len(cg['FIFTEEN_MINUTE'])-fw,1):
    Cmd = normalized_cmd[i-pw:i+fw]
    conso_w = np.mean(cw['eau_final.actual'][i-pw:i+fw])
    conso_g = cg['gaztot.actual'][i-pw:i+fw]
    time_w = cg['FIFTEEN_MINUTE'][i-pw:i+fw]
    temp_ext = []
    for k in time_w:
        #c'est lent mais normal c'est fait exprès pcq si on va trop vite on risque de se faire ban de l'API de météo va te faire un kfé frérot ça va prendre +/- 10 min
        #update : j'ai opti le truc en fait t'as pas le temps pour un kfé

        if k.date() not in temp_today:
            time.sleep(0.02)
            temp = get_temp(k)
            temp_today[k.date()]=temp

        temp_ext.append(temp_today[k.date()][k.hour])
    seg = segment(Cmd,temp_ext,time_w,conso_w,conso_g,pw)
    n_data.append(seg)
    clear_output()
    print(f"😑wala j'y travaille : {float(len(n_data)) /float(len(cg['FIFTEEN_MINUTE'])-fw-pw)}")

😑wala j'y travaille : 1.0


In [18]:
#vérification de la taille
print(len(cg['FIFTEEN_MINUTE'])-fw-pw)
print(len(n_data))
print(n_data[0].getdata())

3911
3911
(array([[63.3999938964844, 66.2000122070313, 60.6000061035156,
        66.3999938964844, 65.6000061035156, 66.3999938964844,
        64.8000183105469, 60.2000122070313, 65.2000122070313,
        65.6000061035156, 72.3999938964844, nan, nan, nan, nan],
       [63.3999938964844, 66.2000122070313, 60.6000061035156,
        66.3999938964844, 65.6000061035156, 66.3999938964844,
        64.8000183105469, 60.2000122070313, 65.2000122070313,
        65.6000061035156, 72.3999938964844, 0.0, 0.0, 0.0, 0.0],
       [63.3999938964844, 66.2000122070313, 60.6000061035156,
        66.3999938964844, 65.6000061035156, 66.3999938964844,
        64.8000183105469, 60.8000183105469, 65.2000122070313,
        65.6000061035156, 72.3999938964844, 0.0, 0.0, 0.0, 0.0],
       [63.3999938964844, 66.2000122070313, 60.6000061035156, 67.0,
        66.2000122070313, 66.3999938964844, 64.8000183105469,
        60.8000183105469, 65.2000122070313, 65.6000061035156,
        72.3999938964844, 0.0, 0.0, 0.0, 0.0

# processing de l'output #

## fonction de recherche de segments similaires ##

In [19]:
dataset=n_data.copy()
def search_sim(input,dataset,threshold):
    sim_ind = []
    list_comp=[1,2,3,4]#élément discriminants pour la similitude de plusieurs situations
    for i in range(len(dataset)):
        a = []
        b = []
        for j in list_comp:
            if type(input.getdata()[j]) == list:
                for k in range(len(input.getdata()[j])):
                    a.append(dataset[i].getdata()[j][k])
                    b.append(input.getdata()[j][k])
            else:
                a.append(dataset[i].getdata()[j])
                b.append(input.getdata()[j])
        a = np.array(a)
        b = np.array(b)
        if np.linalg.norm(a-b)<threshold:
            sim_ind.append(i)
    return sim_ind

print(search_sim(dataset[0],dataset,5))

[0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 924, 926, 928, 930, 934, 1386, 1388, 1390, 1392, 1394, 1395, 1396, 1401, 1482, 1487, 1489, 1492, 1956, 1960, 2035, 2039, 2040, 2042, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2053, 2054, 2790, 2900, 2901]


## fonction d'enregistrement sous .pt ##

In [28]:
import torch
def savefile(input,output,pw):
    path = input.gettime()[pw].strftime('%y-%m-%d %H:%M:%S')
    input = input.flatten()
    input = np.array(input)
    if isinstance(input, np.ndarray):
        input_arr = np.array(input, dtype=np.float32)
        input = torch.from_numpy(input_arr)
    if not torch.isnan(input).any():

        output_arr = np.array(output, dtype=np.float32)
        output = torch.from_numpy(output_arr)
        if not torch.isnan(output).any():
            print(output)
            path = path.replace(' ', '')
            path = path.replace('-', '_')
            path = path.replace(':', '_')

            data = {
            "input": input,
            "output": output
            }
            root = "./dataset/"
            path = root+path

            # Sauvegarde dans un fichier .pt
            torch.save(data, f"{path}.pt")

            print(f"Fichier '{path}.pt' enregistré avec succès.")
    else:
        print("ouille")
print(dataset[0].getdata()[0][-1])
print(dataset[0].getdata())
savefile(dataset[1],dataset[1].getdata()[0][-1],pw)

[64.0 67.0 61.2000122070313 67.0 66.2000122070313 67.0 65.3999938964844
 61.6000061035156 65.8000183105469 66.3999938964844 73.2000122070313 0.0
 0.0 0.0 0.0]
(array([[63.3999938964844, 66.2000122070313, 60.6000061035156,
        66.3999938964844, 65.6000061035156, 66.3999938964844,
        64.8000183105469, 60.2000122070313, 65.2000122070313,
        65.6000061035156, 72.3999938964844, nan, nan, nan, nan],
       [63.3999938964844, 66.2000122070313, 60.6000061035156,
        66.3999938964844, 65.6000061035156, 66.3999938964844,
        64.8000183105469, 60.2000122070313, 65.2000122070313,
        65.6000061035156, 72.3999938964844, 0.0, 0.0, 0.0, 0.0],
       [63.3999938964844, 66.2000122070313, 60.6000061035156,
        66.3999938964844, 65.6000061035156, 66.3999938964844,
        64.8000183105469, 60.8000183105469, 65.2000122070313,
        65.6000061035156, 72.3999938964844, 0.0, 0.0, 0.0, 0.0],
       [63.3999938964844, 66.2000122070313, 60.6000061035156, 67.0,
        66.20001220

In [29]:
os.remove('./dataset/25_07_0705_00_00.pt')

FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable: './dataset/25_07_0705_00_00.pt'

## création des segments pr tout le dataset ##

In [30]:
print(len(dataset))
for i in range(len(dataset)):
    sim_ind = search_sim(dataset[i],dataset,5)
    for j in sim_ind:
        min = dataset[i].getdata()[1]
        if dataset[j].getdata()[1]<min:
            min = dataset[j].getdata()[1]
            indx = j


    chng = dataset[indx].getdata()[0][pw][-4:]
    input = dataset[i]
    savefile(input,abs(chng),pw)


3911
ouille
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0705_15_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0705_30_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0705_45_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0706_00_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0706_15_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0706_30_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0706_45_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0707_15_00.pt' enregistré avec succès.
tensor([0.0000, 0.6981, 0.6982, 0.0000])
Fichier './dataset/25_07_0707_30_00.pt' enregistré avec succès.
tensor([0.5012, 0.7024, 0.6995, 0.4666])
Fi

## appel d'un segment pour tester ##

In [31]:
import torch
test = torch.load('./dataset/25_07_2016_45_00.pt',weights_only=False)
print(test)
print(len(test['input']))
print(len(test['output']))
print(test['input'][263:283])

{'input': tensor([ 62.6000,  65.4000,  59.2000,  65.6000,  64.2000,  65.0000,  64.0000,
         61.6000,  64.4000,  64.2000,  71.0000,   0.0000,   0.7024,   0.6982,
          0.4666,  62.6000,  64.8000,  59.2000,  65.0000,  63.4000,  65.0000,
         63.4000,  59.6000,  63.8000,  64.2000,  71.0000,   0.0000,   0.7024,
          0.6982,   0.4666,  62.6000,  65.4000,  59.2000,  65.6000,  64.2000,
         65.0000,  64.0000,  60.2000,  63.8000,  64.2000,  71.0000,   0.0000,
          0.7024,   0.6982,   0.4666,  62.6000,  65.4000,  59.8000,  65.6000,
         63.4000,  65.0000,  64.0000,  59.6000,  63.8000,  64.2000,  71.0000,
          0.0000,   0.7024,   0.6982,   0.4666,  62.6000,  64.8000,  59.8000,
         65.0000,  64.2000,  65.0000,  63.4000,  59.6000,  63.8000,  64.2000,
         71.0000,   0.0000,   0.7024,   0.6982,   0.4666,  62.0000,  64.8000,
         59.2000,  65.0000,  63.4000,  65.0000,  63.4000,  58.8000,  63.8000,
         64.2000,  71.0000,   0.0000,   0.7024,   0.69

[2, 3]
